In [ ]:
import pandas as pd
import holoviews as hv
import geoviews as gv
from bokeh.sampledata.airport_routes import airports, routes

hv.extension('bokeh')

## Define data

In [ ]:
# Count the number of connections from each airport
counts = routes.groupby('SourceID')[['Stops']].count().reset_index().rename(columns={'Stops': 'Connections'})
airports_df = pd.merge(airports, counts, left_on='AirportID', right_on='SourceID', how='left')

# Select only US mainland airports & convert from Mercator to Latitudes/Longitudes
airport_points = gv.Points(airports_df, ['Longitude', 'Latitude']).select(Longitude=(-170, -50), Latitude=(0, 50))

# Declare nodes, graph and tiles
nodes = gv.Nodes(airport_points, ['Longitude', 'Latitude', 'AirportID'],
                 ['Name', 'City', 'Connections'])
graph = gv.Graph((routes, nodes), ['SourceID', 'DestinationID'], ['Source', 'Destination'])
tiles = gv.WMTS('https://maps.wikimedia.org/osm-intl/{Z}/{X}/{Y}@2x.png')

# Select 50 busiest airports
busiest = list(routes.groupby('SourceID').count().sort_values('Stops').iloc[-50:].index.values)
busiest_airports = graph.select(AirportID=busiest, selection_mode='nodes')

## Plot

In [ ]:
%%opts Graph [width=800 height=600] (edge_selection_line_color='black' edge_hover_line_color='red')
%%opts Graph (edge_line_width=1 edge_line_alpha=0.01 edge_nonselection_line_alpha=0.01)
tiles * busiest_airports